### Важно:
Пожалуйста, поддерживайте ваш код в хорошем состоянии, пишите комментарии, убирайте бесполезные ячейки, пишите модели в специально отведенных модулях. Проверяющие могут **НА СВОЕ УСМОТРЕНИЕ** снижать баллы за:

1. Говнокод
2. Неэффективные решения
3. Вермишель из ячеек в тетрадке
4. Все остальное что им не понравилось

#### (0 - 0.05 балла):

За использование логгеров типа wandb/comet/neptune и красивую сборку этой домашки в виде графиков/картинок в этих логгерах мы будем выдавать бонусные баллы.

Решением домашки является архив с использованными тетрадками/модулями, а так же **.pdf** файл с отчетом по проделанной работе по каждому пункту задачи. В нем необходимо описать какие эксперименты вы производили чтобы получить результат который вы получили, а так же обосновать почему вы решили использовать штуки которые вы использовали (например, дополнительные лоссы для стабилизации, разные виды потоков, разные хаки для вае)


In [ ]:
import os

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

from tqdm.notebook import tqdm as tqdm


%load_ext autoreload

%autoreload 2

! pip install wandb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! gdown --id 1F96x4LDbsTZGMMq81fZr7aduJCe8N95O

import zipfile

path_to_zip = "/content/celeba.zip"


with zipfile.ZipFile(path_to_zip, 'r') as file:
    file.extractall(path='/content')

Downloading...
From: https://drive.google.com/uc?id=1F96x4LDbsTZGMMq81fZr7aduJCe8N95O
To: /content/celeba.zip
2.73GB [00:37, 73.6MB/s]


In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

size = 32
celeba_transforms = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
    



In [16]:

celeba = torchvision.datasets.CelebA('/content/celeba',
                                           transform=celeba_transforms,
                                           download=False)

n = len(celeba)
t = int(n * 0.95)
train_set, val_set = torch.utils.data.random_split(celeba, [t, n - t])

b_s = 16

dataloader = torch.utils.data.DataLoader(train_set, b_s, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_set, 1, shuffle=False)

В этой домашней работе вам предлагается повторить результаты статьи VAE+NF (https://arxiv.org/pdf/1611.05209.pdf).

Основная часть домашнего задания - чтение статьи и повторение результатов, поэтому обязательно прочитайте не только ее, но и другие основные статьи про потоки того времени:

1. https://arxiv.org/abs/1505.05770
2. https://arxiv.org/abs/1605.08803
3. https://arxiv.org/abs/1705.07057
4. http://arxiv.org/abs/1807.03039




### Задача 1 (0.1 балла, но если не сделаете, за всю домашку ноль):

Для начала предлагаю попробовать обучить обычный VAE на Celeba до нормального качества, померить FID и запомнить для будущего сравнения


In [7]:
import wandb

project = 'GM HW3'

In [ ]:
# Simple function for wandb logging
import random
from calculate_fid import calculate_fid

def generate_example(model, loader):
    model.eval()

    rand_index = random.randrange(len(loader.dataset))
    image, label = loader.dataset[rand_index]
    
    device = model.device
    image = image.to(device).unsqueeze(0)
    
    with torch.no_grad():
        reconstructed_image = model.generate(image)[0]

    image = image[0]
    
    image = ((image.permute(1, 2, 0) + 1) / 2).cpu().numpy()
    reconstructed_image = ((reconstructed_image.permute(1, 2, 0) + 1) / 2).cpu().numpy()
    

    example = {'input image': wandb.Image(image),
               'reconstructed image': wandb.Image(reconstructed_image)
               }
    
    return example

def train(image, model, optimizer):
    model.train()

    optimizer.zero_grad()

    out, mu, logvar = model(image)
    loss, rec_loss, kld = loss_function(out, image, mu, logvar)
    
    loss.backward()
    optimizer.step()

    return loss, rec_loss, kld


def process_epoch(epoch, loader, model, optimizer):
    losses = []
    rec_losses = []
    kld_losses = []
    device = model.device
    for i, (image, _) in tqdm(enumerate(loader), desc=f"trainloop: {epoch}", leave=False):
        image = image.to(device)
        
        loss, rec, kld = train(image, model, optimizer)
        losses.append(loss.item())
        rec_losses.append(rec.item())
        kld_losses.append(kld.item())
        
        if (i + 1) % 50 == 0:
            example = generate_example(model, loader)
            l = np.mean(losses)
            rec_loss = np.mean(rec_losses)
            kld_loss = np.mean(kld_losses)
            losses = []
            rec_losses = []
            kld_losses = []
                    
            example.update({'loss': l / len(image),
                            'rec loss': rec_loss / len(image),
                            'KLD': kld_loss / len(image)
                            })
            
            if (i + 1) % 2000 == 0:
                fid = calculate_fid(val_dataloader, model, fid_model)

                example.update({'FID': fid
                                })

            wandb.log(example)



In [ ]:
from model import VAE, loss_function

model = VAE(device, hidden_dim=128)
optim = torch.optim.Adam(model.parameters(), lr=0.001)
PATH = "/content/drive/MyDrive/"

In [ ]:
name = 'VAE_5' 

wandb.init(project=project, name=name);
# wandb.watch(model);


In [8]:
from inception import InceptionV3

fid_model = InceptionV3().to(device)

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth


In [ ]:
epochs = 10
for i in range(1, epochs + 1):
    process_epoch(i, dataloader, model, optim)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(make_grid([model.generate(val_set[i][0].unsqueeze(0).to(device))[0].cpu() for i in range(25)], nrow=5).permute(1,2,0) * 0.5 + 0.5)
plt.show()

Ну да, заблюренные, но об этом и говорили

### Задача 2 (0.3 балла, но если не сделаете, за всю домашку max 0.1 за прошлый пункт):

После этого попробуем обучить обычный NF на Celeba до нормального качества, померить FID и запомнить для будущего сравнения

В качестве потока можно использовать все что вы хотите, Coupling/Autoregressive/Linear слои, любые трансформации. 

Можно использовать как и сверточные потоки, так и линейные (развернув селебу в один вектор)

In [9]:
def calc_loss(log_p, logdet, image_size, n_bins):
    # log_p = calc_log_p([z_list])
    n_pixel = image_size * image_size * 3

    loss = -np.log(n_bins) * n_pixel
    loss = loss + logdet + log_p

    return (
        (-loss / (np.log(2) * n_pixel)).mean(),
        (log_p / (np.log(2) * n_pixel)).mean(),
        (logdet / (np.log(2) * n_pixel)).mean(),
    )



In [10]:
# Simple function for wandb logging
import random
from calculate_fid import calculate_fid


def generate_example_nf(model, loader):
    model.eval()

    rand_index = random.randrange(len(loader.dataset))
    image, label = loader.dataset[rand_index]
    
    device = next(model.parameters()).device
    image = image.to(device).unsqueeze(0)
    
    with torch.no_grad():
        reconstructed_image = model.sample(1, 32)[0]
    
    reconstructed_image = torch.clamp(reconstructed_image, -1, 1)
    reconstructed_image = ((reconstructed_image.permute(1, 2, 0) + 1) / 2).cpu().numpy()
    

    example = {
               'reconstructed image': wandb.Image(reconstructed_image)
               }
    
    return example

def train_nf(image, model, optimizer):
    model.train()
    optimizer.zero_grad()

    image = image * 255

    if n_bits < 8:
        image = torch.floor(image / 2 ** (8 - n_bits))

    image = image / n_bins - 0.5

    log_p, logdet, _ = model(image + torch.rand_like(image) / n_bins)

    logdet = logdet.mean()
    loss, log_p, log_det = calc_loss(log_p, logdet, size, n_bins)

    loss.backward()
    optimizer.step()

    return loss, log_p, log_det


def process_epoch_nf(epoch, loader, model, optimizer):
    best_loss = 1000000.
    losses = []
    log_ps = []
    log_dets = []
    device = next(model.parameters()).device
    for i, (image, _) in tqdm(enumerate(loader), desc=f"trainloop: {epoch}", leave=False):
        image = image.to(device)
        
        loss, log_p, log_det = train_nf(image, model, optimizer)
        losses.append(loss.item())
        log_ps.append(log_p.item())
        log_dets.append(log_det.item())
        
        if (i + 1) % 20 == 0:
            example = generate_example_nf(model, loader)
            l = np.mean(losses)

                    
            example.update({'loss': np.mean(losses) / len(image),
                            'Log P': np.mean(log_ps) / len(image),
                            'Log Det': np.mean(log_dets) / len(image)
                            })
            

            losses = []
            rec_losses = []
            kld_losses = []

            if (i + 1) % 400 == 0:
                fid = calculate_fid(image, model, fid_model)

                example.update({'FID': fid
                                })

            
            wandb.log(example)



In [11]:
n_bits = 5
n_bins = 2.0 ** n_bits

In [24]:
name = 'NF 32 big' 

wandb.init(project=project, name=name);
# wandb.watch(model);


loss,0.14962
Log P,-0.06323
Log Det,0.21207
_runtime,913
_timestamp,1616668155
_step,303
FID,224.61697


loss,█▆▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Log P,▁▂▃▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
Log Det,▁▃▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
FID,█▇▇▇▆▅▄▃▅▄▃▁▂▂▁


In [25]:
from glow import Glow


model = Glow(3, 16, 4).to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.0001)
PATH = "/content/drive/MyDrive/"

In [ ]:
epochs = 10
for i in range(1, epochs + 1):
    process_epoch_nf(i, dataloader, model, optim)

In [ ]:
!nvidia-smi

In [ ]:
!

### Задача 3 (0.6 балла):

Попробуйте повторить архитектуру VAPNEV из https://arxiv.org/pdf/1611.05209.pdf. Сравните качество (FID) между тремя разными моделями

Здесь вы можете использовать VAE и NF из предыдущих пунктов, необходимо только понять как они совмещаются в оригинальной статье

В отчете напишите, почему по вашему мнению такой подход будет лучше (или может быть хуже) чем обычный VAE?



### Бонусная задача (0.2 балла):

Найдите, реализуйте и сравните с предыдущими моделями еще один интересный способ совмещения NF и VAE

##### Подсказки:

1. Если вы учите на колабе или на наших машинках, вероятнее всего что обучение будет очень долгим на картинках 256х256. Никто не мешает уменьшить разрешение, главное чтобы было видно что генерация выучились и качество было ок

2. Вы можете сделать ваш VAE/NF/VAPNEV условным, придумав как вы будете передавать в него conditional аттрибуты селебы

3. Не забывайте про аугментации


